In [1]:
import os, sys
sys.path.append("/workspace/Mask3D_adapted")
import re, os, json
from pathlib import Path
from collections import OrderedDict
import os.path as osp
import numpy as np
import pandas as pd
import open3d as o3d
from fire import Fire
from natsort import natsorted
from loguru import logger

from datasets.preprocessing.base_preprocessing import BasePreprocessing
from utils.point_cloud_utils import load_ply_with_normals

from datasets.scannetpp.scannetpp_constants import (
    CLASS_IDS, VALID_CLASS_IDS, CLASS_LABELS, INSTANCE_LABELS, LABEL2ID)

splits = {'train': "nvs_sem_train.txt",
          'val': "nvs_sem_val.txt",
          'test': "sem_test.txt"}
def filter_map_classes(mapping, count_thresh, count_type, mapping_type):
    mapping = mapping[mapping[count_type] >= count_thresh]
    if mapping_type == "semantic":
        map_key = "semantic_map_to"
    elif mapping_type == "instance":
        map_key = "instance_map_to"
    else:
        raise NotImplementedError
    # create a dict with classes to be mapped
    # classes that don't have mapping are entered as x->x
    # otherwise x->y
    map_dict = OrderedDict()

    for i in range(mapping.shape[0]):
        row = mapping.iloc[i]
        class_name = row["class"]
        map_target = row[map_key]

        # map to None or some other label -> don't add this class to the label list
        try:
            if len(map_target) > 0:
                # map to None -> don't use this class
                if map_target == "None":
                    pass
                else:
                    # map to something else -> use this class
                    map_dict[class_name] = map_target
        except TypeError:
            # nan values -> no mapping, keep label as is
            if class_name not in map_dict:
                map_dict[class_name] = class_name

    return map_dict
def downsample(points, sem_gt, inst_gt, voxel_size=0.025):
    coords = points[:, :3]
    colors = points[:, 3:6]
    normals = points[:, 6:9]
    total_downsample_points = 0
    points_list = []
    colors_list = []
    normals_list = []
    sem_list = []
    inst_list = []
    insts = np.unique(inst_gt)
    for inst in insts:
        inst_mask = inst_gt == inst
        inst_coords = coords[inst_mask]
        inst_colors = colors[inst_mask]
        inst_normals = normals[inst_mask]
        sem_id = sem_gt[inst_mask][0]
        
        inst_pcd = o3d.geometry.PointCloud()
        inst_pcd.points = o3d.utility.Vector3dVector(inst_coords)
        inst_pcd.colors = o3d.utility.Vector3dVector(inst_colors)
        downinst_pcd = inst_pcd.voxel_down_sample(voxel_size=voxel_size)
        
        points_list.append(np.asarray(downinst_pcd.points))
        colors_list.append(np.asarray(downinst_pcd.colors))
        sem_list.append(np.full((len(downinst_pcd.points),), sem_id))
        inst_list.append(np.full((len(downinst_pcd.points),), inst))
        total_downsample_points += len(downinst_pcd.points)
        
        # Create an array to hold the downsampled normals
        downsampled_inst_normals = []
        pcd_tree = o3d.geometry.KDTreeFlann(inst_pcd)
        # Iterate over downsampled points
        for point in downinst_pcd.points:
            # Find the nearest neighbor in the original point cloud
            [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
            # Get the corresponding normal
            downsampled_inst_normals.append(inst_normals[idx[0]])
        normals_list.append(np.asarray(downsampled_inst_normals))
                
    points_dsped = np.concatenate(points_list, axis=0)
    colors_dsped = np.concatenate(colors_list, axis=0)
    normals_dsped = np.concatenate(normals_list, axis=0)
    sem_dsped = np.concatenate(sem_list, axis=0)
    inst_dsped = np.concatenate(inst_list, axis=0)
    return points_dsped, colors_dsped, normals_dsped, sem_dsped, inst_dsped

class ScannetppPreprocessing(BasePreprocessing):
    def __init__(
        self,
        data_dir: str = "./data/raw/scannetpp/scannetpp",
        save_dir: str = "./data/processed/scannetpp",
        modes: tuple = ("train", "val"),
        n_jobs: int = -1,
        ignore_index: int = 0
    ):
        super().__init__(data_dir, save_dir, modes, n_jobs)
        # meta data
        self.ignore_index = ignore_index
        self.create_label_database(data_dir)
        label_mapping = pd.read_csv(
            osp.join(data_dir, "metadata", "semantic_benchmark", "map_benchmark.csv"))
        self.label_mapping = filter_map_classes(
            label_mapping, count_thresh=0, count_type="count", mapping_type="semantic"
        )
        
        for mode in self.modes:
            split_filename = splits[mode]
            split_txt = osp.join(data_dir, "splits", split_filename)
            with open(split_txt, "r") as f:
                # read the scan names without the newline character
                scans = [line.strip() for line in f]   
            folders = []
            for scan in scans:
                scan_folder = osp.join(data_dir, "data", scan)
                folders.append(scan_folder)
            self.files[mode] = natsorted(folders)

    def create_label_database(self, data_dir):
        label_database = {}
        for row_id, class_id in enumerate(CLASS_IDS):
            label_database[class_id] = {
                "name": CLASS_LABELS[row_id],
                "validation": class_id in VALID_CLASS_IDS,
            }
        self._save_yaml(
            self.save_dir / "label_database.yaml", label_database
        )
        return label_database

    def process_file(self, folderpath, mode):
        """process_file.

        Please note, that for obtaining segmentation labels ply files were used.

        Args:
            folderpath: path to the scan folder
            mode: train, test or validation

        Returns:
            filebase: info about file
        """
        scan_id = osp.basename(folderpath)
        mesh_file = osp.join(folderpath, "scans/mesh_aligned_0.05.ply")
        segment_file = osp.join(folderpath, "scans/segments.json")
        anno_file = osp.join(folderpath, "scans/segments_anno.json")
        filebase = {
            "filepath": folderpath,
            'raw_filepath': str(folderpath),
            "scene": scan_id,
            "mesh_file": mesh_file,
            'segment_file': segment_file,
            'anno_file': anno_file,
        }
        # reading both files and checking that they are fitting
        coords, features, _ = load_ply_with_normals(mesh_file)
        file_len = len(coords)
        filebase["file_len"] = file_len
        points = np.hstack((coords, features))
    
        # get segment ids and instance ids
        with open(segment_file) as f:
            segments = json.load(f)
        # load anno = (instance, groups of segments)
        with open(anno_file) as f:
            anno = json.load(f)
            
        seg_indices = np.array(segments["segIndices"], dtype=np.uint32)
        num_vertices = len(seg_indices)
        assert num_vertices == points.shape[0]
        semantic_gt = np.ones(num_vertices, dtype=np.int32) * self.ignore_index
        instance_gt = np.ones(num_vertices, dtype=np.int32) * self.ignore_index
        assigned = np.zeros(num_vertices, dtype=bool)
        for idx, instance in enumerate(anno["segGroups"]):
            label = instance["label"]
            # remap label
            instance["label"] = self.label_mapping.get(label, None)
            instance["label_index"] = LABEL2ID.get(instance["label"], self.ignore_index)
            if instance["label_index"] == self.ignore_index:
                continue
            # get all the vertices with segment index in this instance
            # and max number of labels not yet applied
            # mask = np.isin(seg_indices, instance["segments"]) & (labels_used < 3)
            mask = np.zeros(num_vertices, dtype=bool)
            mask[instance["segments"]] = True
            mask = np.logical_and(mask, ~assigned)
            size = mask.sum()
            if size == 0:
                continue
            # get semantic labels
            semantic_gt[mask] = instance["label_index"]
            assigned[mask] = True
            
            # store all valid instance (include ignored instance)
            if instance["label"] in INSTANCE_LABELS:
                instance_gt[mask] = instance["objectId"]
                
            gt_label_inspect = instance["label_index"] * 1000 + instance["objectId"] + 1
            if gt_label_inspect < 0:
                print("     instance[label_index]: {}; instance[objectId]: {}".format(["label_index"], instance["objectId"]))
                
        # downsample the points
        coords, colors, normals, sem_gt, inst_gt = downsample(
            points, semantic_gt, instance_gt)
        segments_placeholder = np.zeros_like(sem_gt)
        points = np.hstack((coords, colors, normals, sem_gt[..., None], inst_gt[..., None],
                            segments_placeholder[..., None]))
        ## save the downsampled points
        processed_filepath = (
            self.save_dir / mode / f"{scan_id}.npy"
        )
        if not processed_filepath.parent.exists():
            processed_filepath.parent.mkdir(parents=True, exist_ok=True)
        np.save(processed_filepath, points.astype(np.float32))
        filebase["filepath"] = str(processed_filepath)
        
        gt_labels = sem_gt * 1000 + inst_gt + 1
        processed_gt_filepath = (
            self.save_dir
            / "instance_gt"
            / mode
            / (scan_id + ".txt")
        )
        if not processed_gt_filepath.parent.exists():
            processed_gt_filepath.parent.mkdir(parents=True, exist_ok=True)
        np.savetxt(processed_gt_filepath, gt_labels.astype(np.int32), fmt="%d")
        filebase["instance_gt_filepath"] = str(processed_gt_filepath)

        filebase["color_mean"] = [
            float((colors[:, 0] / 255).mean()),
            float((colors[:, 1] / 255).mean()),
            float((colors[:, 2] / 255).mean()),
        ]
        filebase["color_std"] = [
            float(((colors[:, 0] / 255) ** 2).mean()),
            float(((colors[:, 1] / 255) ** 2).mean()),
            float(((colors[:, 2] / 255) ** 2).mean()),
        ]
        return filebase

    def compute_color_mean_std(
        self,
        train_database_path: str = "./data/processed/scannetpp/train_database.yaml",
    ):
        train_database = self._load_yaml(train_database_path)
        color_mean, color_std = [], []
        for sample in train_database:
            color_std.append(sample["color_std"])
            color_mean.append(sample["color_mean"])

        color_mean = np.array(color_mean).mean(axis=0)
        color_std = np.sqrt(np.array(color_std).mean(axis=0) - color_mean**2)
        feats_mean_std = {
            "mean": [float(each) for each in color_mean],
            "std": [float(each) for each in color_std],
        }
        self._save_yaml(self.save_dir / "color_mean_std.yaml", feats_mean_std)

    @logger.catch
    def fix_bugs_in_labels(self):
        pass
    
    def joint_database(self, train_modes=["train", "val"]):
        joint_db = []
        for mode in train_modes:
            joint_db.extend(
                self._load_yaml(self.save_dir / (mode + "_database.yaml"))
            )
        self._save_yaml(
            self.save_dir / "train_validation_database.yaml", joint_db
        )


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
data_dir="/workspace/Mask3D_adapted/data/raw/scannetpp" 
save_dir="/workspace/Mask3D_adapted/data/processed/scannetpp" 

preprocessor = ScannetppPreprocessing(data_dir, save_dir)

In [7]:
folder_path = "/workspace/Mask3D_adapted/data/raw/scannetpp/data/0d2ee665be"

preprocessor.process_file(folder_path, "val")


{'filepath': '/workspace/Mask3D_adapted/data/processed/scannetpp/val/0d2ee665be.npy',
 'raw_filepath': '/workspace/Mask3D_adapted/data/raw/scannetpp/data/0d2ee665be',
 'scene': '0d2ee665be',
 'mesh_file': '/workspace/Mask3D_adapted/data/raw/scannetpp/data/0d2ee665be/scans/mesh_aligned_0.05.ply',
 'segment_file': '/workspace/Mask3D_adapted/data/raw/scannetpp/data/0d2ee665be/scans/segments.json',
 'anno_file': '/workspace/Mask3D_adapted/data/raw/scannetpp/data/0d2ee665be/scans/segments_anno.json',
 'file_len': 1082845,
 'instance_gt_filepath': '/workspace/Mask3D_adapted/data/processed/scannetpp/instance_gt/val/0d2ee665be.txt',
 'color_mean': [0.5325205791511989, 0.5141894949042368, 0.4850900186942797],
 'color_std': [0.3185788923604686, 0.30037131115908294, 0.27732087317186194]}

In [5]:
import os, numpy as np
gt_folder = "/workspace/Mask3D_adapted/data/processed/scannetpp/instance_gt/validation"
files = os.listdir(gt_folder)

In [7]:
sem_list = []
for file in files:
    sem = np.loadtxt(os.path.join(gt_folder, file))
    sem = (sem/1000).astype(np.int32)
    sem_unqiue = np.unique(sem)
    sem_list.extend(sem_unqiue)


In [11]:
sem_arr = np.unique(np.array(sem_list))

In [13]:
sem_arr_valid = sem_arr[5:]

In [23]:
CLASS_LABELS = ('table', 'door',
'ceiling lamp', 'cabinet', 'blinds', 'curtain', 'chair', 'storage cabinet', 'office chair', 'bookshelf', 
'whiteboard', 'window', 'box', 'monitor', 'shelf', 'heater', 'kitchen cabinet', 'sofa', 'bed', 'trash can', 'book',
'plant', 'blanket', 'tv', 'computer tower', 'refrigerator', 'jacket', 'sink', 'bag', 'picture', 'pillow', 'towel',
'suitcase', 'backpack', 'crate', 'keyboard', 'rack', 'toilet', 'printer', 'poster', 'painting', 'microwave', 'shoes',
'socket', 'bottle', 'bucket', 'cushion', 'basket', 'shoe rack', 'telephone', 'file folder', 'laptop', 'plant pot',
'exhaust fan', 'cup', 'coat hanger', 'light switch', 'speaker', 'table lamp', 'kettle', 'smoke detector', 'container',
'power strip', 'slippers', 'paper bag', 'mouse', 'cutting board', 'toilet paper', 'paper towel', 'pot', 'clock',
'pan', 'tap', 'jar', 'soap dispenser', 'binder', 'bowl', 'tissue box', 'whiteboard eraser', 'toilet brush', 
'spray bottle', 'headphones', 'stapler', 'marker'
)
VALID_CLASS_IDS= np.array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32,
33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 
59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 
85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100])

ID_TO_LABEL = {}
LABEL_TO_ID = {}
for i in range(len(VALID_CLASS_IDS)):
    LABEL_TO_ID[CLASS_LABELS[i]] = VALID_CLASS_IDS[i]
    ID_TO_LABEL[VALID_CLASS_IDS[i]] = CLASS_LABELS[i]

In [21]:
for class_id in VALID_CLASS_IDS:
    if class_id not in sem_arr_valid:
        print(CLASS_LABELS[class_id-17])

rack
file folder
stapler


In [29]:
label_check_list = ["marker", "whiteboard eraser", "jar", "smoke detector", "mouse"]
label_check_size = {}
data = {}
for file in files:
    inst = np.loadtxt(os.path.join(gt_folder, file))
    sem = (inst/1000).astype(np.int32)
    data[file] = {
        'inst': inst, 'sem': sem}
for label_check in label_check_list:
    label_id = LABEL_TO_ID[label_check]
    size_list = []

    for file in files:
        inst = data[file]['inst']
        sem = data[file]['sem']
        inst_uniques = np.unique(inst[sem == label_id])
        for inst_id in inst_uniques:
            size_list.append( np.sum(inst == inst_id) )
    label_check_size[label_check] = size_list


In [30]:
label_check_size

{'marker': [17, 16, 54],
 'whiteboard eraser': [15, 55, 76, 63, 46, 51, 63],
 'jar': [28, 22, 27, 41, 18, 37, 25, 43, 10, 14, 22, 26, 23, 33, 37],
 'smoke detector': [62,
  79,
  92,
  52,
  51,
  82,
  85,
  37,
  73,
  19,
  9,
  79,
  75,
  56,
  33,
  55,
  69,
  51,
  66,
  35,
  41,
  52,
  45,
  19,
  47],
 'mouse': [13,
  24,
  38,
  20,
  25,
  24,
  43,
  32,
  26,
  36,
  25,
  41,
  30,
  38,
  34,
  26,
  39,
  29,
  16,
  53,
  26,
  39,
  32,
  35,
  34,
  40,
  25,
  41,
  37,
  29,
  27,
  23,
  23]}